In [38]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import os
from bs4 import BeautifulSoup
import pandas as pd
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException



premier league url
https://www.capology.com/uk/premier-league/salaries/

bundesliga url
https://www.capology.com/de/1-bundesliga/salaries/

la liga url
https://www.capology.com/es/la-liga/salaries/

serie a url
https://www.capology.com/it/serie-a/salaries/

ligue 1 url
https://www.capology.com/fr/ligue-1/salaries/


# Webscraper of capology.com
This webscraper was made to extract contract length and renewals for big five leagues past years. 

### To run the webscraper, simply run the cells with the defined functions and the main loop found in the following Jupyter Notebook cells.

To-do:
1. Make the webscraper more robust to errors
2. Create more functions to make the main loop smaller
3. More precise commenting to explain steps in code



Bug-note: There seems to be some errors with this webscraper which are unsolved as of 30.04.2022.
As the error "WebDriverException: Message: target frame detached" randomly occured, the webdriver was changed to Firefox which avoided the error. 

Another error which have occured is that bs4 may not find the 'tbody' in the table class, however this could be a result of poor understanding of html code.

In [86]:
def open_webdriver(start_url):
    # init webdriver
    driver = webdriver.Firefox()

    # Starting webdriver
    driver.get(start_url)

    # Emulate human behaviour
    time.sleep(random.randint(0, 3))
    return driver

def get_table_data(driver):
    content = driver.find_element(By.CSS_SELECTOR, '#table')
    soup = BeautifulSoup(content.get_attribute('innerHTML'))

    # Extracting the rows
    tbody = soup.find('tbody')
    data = []
    for tr in tbody:
        rows = []
        print("length of tr ", len(tr))
        for td in tr:
            print("length of td ",len(td) )
            rows.append(td.get_text())
            #print(td.get_text())
        data.append(rows)

    df = pd.DataFrame(data)



    # Getting header
    thead = soup.find('thead')

    thead_tr = thead.find_all('tr')[1]
    header = []
    for th in thead_tr:
        header.append(th.get_text())

    df.columns = header

    # Getting season info
    title = driver.find_element(By.CSS_SELECTOR, '#panel > div.content-block > div > div:nth-child(2) > div:nth-child(1) > h4')    
    title_soup = BeautifulSoup(title.get_attribute('innerHTML'))    
    season = title_soup.get_text()[:9]
    df["season"] = season
    
    return df

def data_cleaner(df):
    # Tidying up column names
    columns = ['Player', 'Signed', 'WeeklyGrossEuro' ,'AnnualGrossEuro', 'TotalGrossEuro', 'ExpDate', 'Length', 'Club', 'season' ]
    df.columns = columns
    
    # Cleaning strings to floats
    df["WeeklyGrossEuro"] = df["WeeklyGrossEuro"].apply( lambda x : float(x[1:].replace(",", "")))
    df["AnnualGrossEuro"] = df["AnnualGrossEuro"].apply( lambda x : float(x[1:].replace(",", "")))
    df["Length"] = df["Length"].apply( lambda x : float(x[0]))
    df["TotalGrossEuro"] = df["TotalGrossEuro"].apply( lambda x : float(x[1:].replace(",", "")))
    df["ExpDateYear"] = df["ExpDate"].apply( lambda x : float(x[-4:]))
    return df

## Running Data Scraper

In [87]:
league_key = [
    #["uk", "premier-league"],
   # ["de", "1-bundesliga"],
   # ["es", "la-liga"],
    ["it", "serie-a"],
    ["fr", "ligue-1"]
]
SEASON = ['2021-2022','2020-2021','2019-2020', '2018-2019']#, '2017-2018']


driver = open_webdriver('https://www.capology.com')

# Emulate human behaviour
time.sleep(random.randint(1, 2))

for key in league_key:
    
    # Updating leauge url
    print(key)
    url = 'https://www.capology.com/{}/{}/salaries'.format(key[0], key[1])
    
    driver.get(url)
    
    df = pd.DataFrame()
    for season_id in SEASON:
        # Emulate human behaviour
        time.sleep(random.randint(0, 3))

        print("Selecting season")
        season_select = Select(driver.find_element(By.CSS_SELECTOR, '#panel > div.row.fixedElement.lg-color > ul > li > div.col.s6.m3.l2 > form > select'))
        season_select.select_by_visible_text(season_id) 

        # Emulate human behaviour
        time.sleep(random.randint(1, 2))

        # Select renewals
        print("Selecting renewals")
        season_select = Select(driver.find_element(By.CSS_SELECTOR, '#panel > div.row.fixedElement.lg-color > ul > li > div.col.s6.m3.l3 > form > select'))
        season_select.select_by_visible_text('Renewals')

        # Emulate human behaviour
        time.sleep(random.randint(1, 2))
        
        # Checking if "All"-button is pressed.
        try:
            all_button = driver.find_element(By.XPATH, '/html/body/main/div[3]/div/div[4]/div[2]/div[2]/div[1]/div/div/div/a[6]')
            all_button.click()
            time.sleep(random.randint(1,2))
        except NoSuchElementException as e:
            pass        
        
        
        
        # Select euro
        print("Selecting currency: Euro")
        euro_button = driver.find_element(By.CSS_SELECTOR, '#btn_eur')
        euro_button.click()
        time.sleep(random.randint(1,2))
        
        
        # Extracting and cleaning up data
        print("Extracting data")
        temp_df = get_table_data(driver)
        print("Cleaning data")
        temp_df = data_cleaner(temp_df)
    
        df = pd.concat([df, temp_df])
    df["League"] = key[1]
    
    # Saving each league to a .csv file inside data.
    df.to_csv("data/"+key[1]+".csv") 

['it', 'serie-a']
Selecting season
Selecting renewals
Selecting currency: Euro
Extracting data
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of

Selecting currency: Euro
Extracting data
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1

Selecting season
Selecting renewals
Selecting currency: Euro
Extracting data
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of t

Selecting season
Selecting renewals
Selecting currency: Euro
Extracting data
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of t

['fr', 'ligue-1']
Selecting season
Selecting renewals
Selecting currency: Euro
Extracting data
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of

Selecting season
Selecting renewals
Selecting currency: Euro
Extracting data
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of t

Selecting renewals
Selecting currency: Euro
Extracting data
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of td  1
length of tr  8
length of td  1
length of td  1
length of td  1
length of td